In [1]:
import models
from models import combine_bench_prior, load_priorband_data,add_rel_ranks,rename_algos,create_incumbent, model
import pandas as pd
import numpy as np

algorithm="algorithm";benchmark="bench_prior";seed="seed";time="used_fidelity"
algos=["pb_mutation_dynamic_geometric-default-at-target","random_search","hyperband"]
fs=[24]
f_space=np.linspace(1,max(fs),max(fs)).tolist()
benchmarks=["jahs_cifar10","jahs_colorectal_histology","jahs_fashion_mnist","lcbench-126026","lcbench-167190","lcbench-168330","lcbench-168910","lcbench-189906","cifar100_wideresnet_2048","imagenet_resnet_512","lm1b_transformer_2048","translatewmt_xformer_64"]
label_dict={'random_search': 'RS', 'hyperband': 'HB', 'pb_mutation_dynamic_geometric-default-at-target': 'PB', 'jahs_cifar10': 'JAHS-C10', 'jahs_colorectal_histology': 'JAHS-CH', 'jahs_fashion_mnist': 'JAHS-FM', 'lcbench-126026': 'LC-126026', 'lcbench-167190': 'LC-167190', 'lcbench-168330': 'LC-168330', 'lcbench-168910': 'LC-168910', 'lcbench-189906': 'LC-189906', 'cifar100_wideresnet_2048': 'PD1-Cifar100', 'imagenet_resnet_512': 'PD1-ImageNet', 'lm1b_transformer_2048': 'PD1-LM1B', 'translatewmt_xformer_64': 'PD1-WMT',"random_search_prior":"RS+Prior","bo":"BO","pibo":"PiBO","bohb":"BOHB","priorband_bo":"PriorBand+BO"}

data=pd.read_parquet("priorband_benchPrior_relRanks_f24_meta.parquet")

In [2]:
loss="value"
m0_bar=model(formula=f"{loss}~{algorithm}",data=data)
m0=model(formula=f"{loss}~{algorithm}+(1|{benchmark})",data=data)
m1=model(formula=f"{loss}~{algorithm}+{benchmark}",data=data)

print("Task 1️⃣")
models.glrt(m1,m0_bar,names=["M₁","M₀'"])
print("=> Benchmarks are important in determining algorithms behaviour \n")
print("Task 2️⃣")
models.glrt(m1,m0,names=["M₁","M₀"])
print("=> And they should be a fixed factor")

Task 1️⃣
M₁ (-243759.47) >> M₀' (-269924.24)
Chi-Square: 52329.54021185031, P-Value: 0.0
=> Benchmarks are important in determining algorithms behaviour 

Task 2️⃣
M₁ (-243759.47) >> M₀ (-243867.54)
Chi-Square: 216.12467401474714, P-Value: 0.0
=> And they should be a fixed factor


In [3]:
print("Task 3️⃣ (Relative Ranks)")
loss="rel_rank"
mrel_0_bar=model(formula=f"{loss}~{algorithm}",data=data)
mrel_0=model(formula=f"{loss}~{algorithm}+(1|{benchmark})",data=data)
mrel_1=model(formula=f"{loss}~{algorithm}+{benchmark}",data=data)

models.glrt(mrel_1,mrel_0_bar,names=["M₁-rel","M₀'-rel"])
print("=> Including benchmarks does not yield better model (as random effect introduces overfitting) \n")
models.glrt(mrel_1,mrel_0,names=["M₁-rel","M₀-rel"])
print("=> Still no better model\n")


Task 3️⃣ (Relative Ranks)
boundary (singular) fit: see help('isSingular') 

M₁-rel (-96407.74) == M₀'-rel (-96407.74)
Chi-Square: 8.265487849712372e-09, P-Value: 1.0
=> Including benchmarks does not yield better model (as random effect introduces overfitting) 

M₁-rel (-96407.74) == M₀-rel (-96407.74)
Chi-Square: 1.2456439435482025e-08, P-Value: 1.0
=> Still no better model



In [4]:
print("Task 4️⃣")
loss="value"
m2=model(formula=f"{loss}~{algorithm}+{benchmark}+{algorithm}:{benchmark}",data=data)

models.glrt(m1,m2,names=["M₁","M₂"])
print("=> Interaction effect is significance \n")

loss="rel_rank"
m2_bar=model(formula=f"{loss}~{algorithm}+{algorithm}:{benchmark}",data=data)
models.glrt(mrel_0_bar,m2_bar,["m_rel0'","m2'"])

Task 4️⃣
M₁ (-243759.47) << M₂ (-242526.64)
Chi-Square: 2465.6748767755926, P-Value: 0.0
=> Interaction effect is significance 

m_rel0' (-96407.74) << m2' (-91592.15)
Chi-Square: 9631.1719700807, P-Value: 0.0


In [5]:

print("Task 5️⃣")
for loss in ["value","regret","norm_regret","rel_rank"]:
    list=[]
    for benchmark in ["bench_prior","n_Vars","n_cont_Vars","n_cat_Vars","int"]:
        print(f"\nLoss: {loss}, Benchmark: {benchmark}")
        m3_bar=model(formula=f"{loss}~{algorithm}",data=data)
        m3_bar_bar=model(formula=f"{loss}~{algorithm}+(1|{benchmark})",data=data)
        m3=model(formula=f"{loss}~{algorithm}+{benchmark}",data=data)
        list.append(m3.logLike)
        models.glrt(m3,m3_bar,names=["M₃","M₃'"])
        models.glrt(m3,m3_bar_bar,names=["M₃","M₃''"])
        

Task 5️⃣

Loss: value, Benchmark: bench_prior
M₃ (-243759.47) >> M₃' (-269924.24)
Chi-Square: 52329.54021185031, P-Value: 0.0
M₃ (-243759.47) >> M₃'' (-243867.54)
Chi-Square: 216.12467401474714, P-Value: 0.0

Loss: value, Benchmark: n_Vars
M₃ (-251270.1) >> M₃' (-269924.24)
Chi-Square: 37308.285388170276, P-Value: 0.0
M₃ (-251270.1) << M₃'' (-247691.9)
Chi-Square: 7156.405198226334, P-Value: 0.0

Loss: value, Benchmark: n_cont_Vars
M₃ (-247691.44) >> M₃' (-269924.24)
Chi-Square: 44465.614598759916, P-Value: 0.0
M₃ (-247691.44) >> M₃'' (-247702.86)
Chi-Square: 22.842979190056212, P-Value: 1.7579029557568404e-06

Loss: value, Benchmark: n_cat_Vars
M₃ (-247691.44) >> M₃' (-269924.24)
Chi-Square: 44465.61459876015, P-Value: 0.0
M₃ (-247691.44) >> M₃'' (-247702.86)
Chi-Square: 22.842979190289043, P-Value: 1.7579029555347958e-06

Loss: value, Benchmark: int
M₃ (-268741.67) >> M₃' (-269924.24)
Chi-Square: 2365.158368468052, P-Value: 0.0
M₃ (-268741.67) >> M₃'' (-268750.05)
Chi-Square: 16.7661

In [6]:
print("Task 6️⃣")
loss="value";benchmark="bench_prior"
m4=model(formula=f"{loss}~{algorithm}:{benchmark}+{time}",data=data)
# m4_0=model(formula=f"{loss}~{algorithm}:{benchmark}+{time}+{algorithm}:{benchmark}:{time}+(1|{seed})",data=data)
# Singular fit error
m4_bar=model(formula=f"{loss}~{algorithm}:{benchmark}+{time}+({algorithm}:{benchmark}):{time}",data=data)

models.glrt(m4,m4_bar,names=["m₄","m₄'"])
print("=> Expanded model is better equipped")

Task 6️⃣
m₄ (-241509.12) << m₄' (-232769.63)
Chi-Square: 17478.9785439088, P-Value: 0.0
=> Expanded model is better equipped
